In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import train_test_split


import tensorflow as tf
from keras.models import Sequential,load_model
from keras import layers,Input
from keras.losses import binary_crossentropy
from keras.layers import Dense
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint
import keras.backend as K

2023-11-09 11:46:06.500193: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-09 11:46:08.962870: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


2023-10-25 11:10:49.404383: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
arry=np.array([[[1,11],[2,12],[3,13],[4,14],[5,15],[6,16],[7,17],[8,18],[9,19],[10,20]],[[1,11],[2,12],[3,13],[4,14],[5,15],[6,16],[7,17],[8,18],[9,19],[10,20]]])
# arry=np.array([[[1,2,3,4,5,6,7,8,9,10],[11,12,13,14,15,16,17,18,19,20]],[[1,2,3,4,5,6,7,8,9,10],[11,12,13,14,15,16,17,18,19,20]]])
print(arry.shape)
print(arry)
print(arry.reshape((2,5,4)))

(2, 10, 2)
[[[ 1 11]
  [ 2 12]
  [ 3 13]
  [ 4 14]
  [ 5 15]
  [ 6 16]
  [ 7 17]
  [ 8 18]
  [ 9 19]
  [10 20]]

 [[ 1 11]
  [ 2 12]
  [ 3 13]
  [ 4 14]
  [ 5 15]
  [ 6 16]
  [ 7 17]
  [ 8 18]
  [ 9 19]
  [10 20]]]
[[[ 1 11  2 12]
  [ 3 13  4 14]
  [ 5 15  6 16]
  [ 7 17  8 18]
  [ 9 19 10 20]]

 [[ 1 11  2 12]
  [ 3 13  4 14]
  [ 5 15  6 16]
  [ 7 17  8 18]
  [ 9 19 10 20]]]


In [14]:
state0=np.load('../Data/new/2gsps/state0_her.npy')
state1=np.load('../Data/new/2gsps/state1_her.npy')
output0=np.zeros((state0.shape[0]))
output1=np.ones((state1.shape[0]))
print(state0.shape)

(5717, 8192, 2)


In [15]:
x=np.vstack((state0,state1))
print(x.shape)
y=np.hstack((output0,output1))
print(y.shape)

(11502, 8192, 2)
(11502,)


In [48]:

st,ed=700,2100
x_train, x_test, y_train, y_test = train_test_split(x[:,st:ed,:], y, test_size=0.30, random_state=45)
print(x_train.shape)
print(y_train.sum())

x_train=np.mean(x_train,axis=1)
x_test=np.mean(x_test,axis=1)

# x_train=x_train.reshape((x_train.shape[0],x_train.shape[1]*2))
# x_test=x_test.reshape((x_test.shape[0],x_test.shape[1]*2))
print(x_train.shape)

(8051, 1400, 2)
4065.0
(8051, 2)


## Logistic Regression

In [49]:
# x_trainC=x_train.reshape((x_train.shape[0],x_train.shape[1]*2))
# x_testC=x_test.reshape((x_test.shape[0],x_test.shape[1]*2))
classifier = LogisticRegression(random_state = 42, max_iter=500)
classifier.fit(x_train, y_train)
print(classifier.score(x_test,y_test))
pred = classifier.predict(x_test)

0.799478412054477


## SVM

In [50]:
model=SVC(decision_function_shape='ovo')
model.fit(x_train,y_train)
print(model.score(x_test,y_test))
pred=model.predict(x_test)

0.8009272674587076


## GMM

In [63]:
Y=list(zip(np.append(np.mean(state0[:,st:ed,0],axis=1),np.mean(state1[:,st:ed,0],axis=1)),np.append(np.mean(state0[:,st:ed,1],axis=1),np.mean(state1[:,st:ed,1],axis=1))))
gmm_Y=GaussianMixture(n_components=2,covariance_type='full').fit(Y)
bitstring=np.split(gmm_Y.predict(Y),2)
print('P(0|0)=',len(bitstring[0][bitstring[0]==0])/len(bitstring[0]))
print('P(1|1)=',len(bitstring[1][bitstring[1]==1])/len(bitstring[1]))
X=list(zip(np.mean(state0[:,st:ed,0],axis=1),np.mean(state0[:,st:ed,1],axis=1)))
gmm_Y_1=GaussianMixture(n_components=1,covariance_type='spherical').fit(X)
X=list(zip(np.mean(state1[:,st:ed,0],axis=1),np.mean(state1[:,st:ed,1],axis=1)))
gmm_Y_2=GaussianMixture(n_components=1,covariance_type='spherical').fit(X)

P(0|0)= 0.7995131281516258
P(1|1)= 0.7906451051990958


## NN

In [53]:
def arch():
    model=Sequential()
    model.add(Input(shape=(2), name='Input Layer'))
    model.add(Dense(8,activation='relu', name='HiddenLayer1'))
    model.add(Dense(4,activation='relu', name='HiddenLayer2'))
    model.add(Dense(1,activation='sigmoid', name='OuputLayer'))
    return model
arch().summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 HiddenLayer1 (Dense)        (None, 8)                 24        
                                                                 
 HiddenLayer2 (Dense)        (None, 4)                 36        
                                                                 
 OuputLayer (Dense)          (None, 1)                 5         
                                                                 
Total params: 65 (260.00 Byte)
Trainable params: 65 (260.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [54]:
def bceloss(y_true, y_pred):
    # Clip the prediction value to prevent log(0) error
    y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
    # Calculate the weighted binary cross entropy loss
    loss = -(0.1 * (y_true * K.log(y_pred)) + 0.9*((1 - y_true) * K.log(1 - y_pred)))
    return K.mean(loss, axis=-1)

model=arch()
model.summary
opt=SGD( learning_rate=0.01, momentum=0.9)
model.compile(loss=binary_crossentropy, optimizer='adam', metrics=['accuracy'])
model_path="../Model/NN2/tmp/{epoch:02d}-{val_accuracy:.4f}.h5"
checkpoint = ModelCheckpoint(model_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [55]:
H=model.fit(x_train,y_train,
          validation_split=0.2,
          epochs=100,batch_size=64,
          callbacks=callbacks_list,
          verbose=1)

Epoch 1/100
 47/101 [============>.................] - ETA: 0s - loss: 0.7290 - accuracy: 0.5110 
Epoch 1: val_accuracy improved from -inf to 0.72874, saving model to ../Model/NN2/tmp/01-0.7287.h5
101/101 [==============================] - 1s 3ms/step - loss: 0.6556 - accuracy: 0.6008 - val_loss: 0.5821 - val_accuracy: 0.7287
Epoch 2/100
 56/101 [===============>..............] - ETA: 0s - loss: 0.5640 - accuracy: 0.7812

/global/homes/n/nrvora/.conda/envs/readout/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_accuracy improved from 0.72874 to 0.75109, saving model to ../Model/NN2/tmp/02-0.7511.h5
101/101 [==============================] - 0s 2ms/step - loss: 0.5572 - accuracy: 0.7806 - val_loss: 0.5616 - val_accuracy: 0.7511
Epoch 3/100
 60/101 [================>.............] - ETA: 0s - loss: 0.5410 - accuracy: 0.7789
Epoch 3: val_accuracy improved from 0.75109 to 0.75295, saving model to ../Model/NN2/tmp/03-0.7529.h5
101/101 [==============================] - 0s 3ms/step - loss: 0.5385 - accuracy: 0.7863 - val_loss: 0.5505 - val_accuracy: 0.7529
Epoch 4/100
 87/101 [========================>.....] - ETA: 0s - loss: 0.5262 - accuracy: 0.7872
Epoch 4: val_accuracy improved from 0.75295 to 0.75419, saving model to ../Model/NN2/tmp/04-0.7542.h5
101/101 [==============================] - 0s 2ms/step - loss: 0.5258 - accuracy: 0.7884 - val_loss: 0.5429 - val_accuracy: 0.7542
Epoch 5/100
 65/101 [==================>...........] - ETA: 0s - loss: 0.5199 - accuracy: 0.7861
Epoch 5: 

In [57]:
model=load_model('../Model/NN2/tmp/29-0.7716.h5', compile=False)
count=0
count1=0
y_pred=model.predict(x_test)
for i in range(y_pred.shape[0]):
    if y_test[i]==1:
        count+=1
        if y_pred[i]>=0.5:
            count1+=1
print(count,count1)

108/108 [==============================] - 0s 524us/step
1720 1368


In [6]:
np.save('../x_text.npy',x_test)
np.save('../y_test.npy',y_test)

## GDA

In [20]:
import numpy as np
from scipy.stats import multivariate_normal

class GaussianDiscriminantAnalysis:
    def fit(self, X, y):
        self.classes = np.unique(y)
        self.class_priors = np.array([np.mean(y == c) for c in self.classes])

        self.means = []
        self.cov_matrices = []
        for c in self.classes:
            X_c = X[y == c]
            mean_c = np.mean(X_c, axis=0)
            cov_matrix_c = np.cov(X_c, rowvar=False)
            self.means.append(mean_c)
            self.cov_matrices.append(cov_matrix_c)

    def predict(self, X):
        posteriors = []

        for c in self.classes:
            class_prior = self.class_priors[int(c)]
            mean = self.means[int(c)]
            cov_matrix = self.cov_matrices[int(c)]
            mvn = multivariate_normal(mean=mean, cov=cov_matrix)
            posterior = class_prior * mvn.pdf(X)
            posteriors.append(posterior)

        posteriors = np.array(posteriors).T
        predicted_labels = np.argmax(posteriors, axis=1)

        return predicted_labels



# Initialize and train GDA
gda = GaussianDiscriminantAnalysis()
gda.fit(x_train, y_train)

# Make predictions
y_pred = gda.predict(x_train)
# Calculate accuracy
accuracy = np.mean(y_pred == y_train)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 96.15%


## RNN

In [48]:
min,max= -(2**16),(2**16)
print('Min: %.3f, Max: %.3f' % (x.min(), x.max()))
x= (x-min)/(max-min)
print('Min: %.3f, Max: %.3f' % (x.min(), x.max()))

Min: -32291.412, Max: 28569.142
Min: 0.254, Max: 0.718


In [49]:
st,ed=700,2700
x_train, x_test, y_train, y_test = train_test_split(x[:,st:ed,:], y, test_size=0.30, random_state=45)
x_train=x_train.reshape((x_train.shape[0],200,20))
x_test=x_test.reshape((x_test.shape[0],200,20))
print(x_train.shape)
print(y_train.sum())

(8400, 200, 20)
4233.0


In [61]:
def rnn():
    model=Sequential()
    model.add(Input(shape=(200,20)))
    model.add(layers.LSTM(units=16, return_sequences=True))
    model.add(layers.LSTM(units=8, return_sequences=True))
    model.add(layers.LSTM(units=4, return_sequences=False))
#     model.add(layers.Dense(4,activation='relu'))
    model.add(layers.Dense(1,activation='sigmoid'))
    return model

In [62]:
model=rnn()
model.summary()
opt=SGD( learning_rate=0.001, momentum=0.9)
model.compile(loss=binary_crossentropy, optimizer='adam', metrics=['accuracy'])
model_path="../Model/RNN_new/{epoch:02d}-{val_accuracy:.4f}.h5"
checkpoint = ModelCheckpoint(model_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_29 (LSTM)              (None, 200, 16)           2368      
                                                                 
 lstm_30 (LSTM)              (None, 200, 8)            800       
                                                                 
 lstm_31 (LSTM)              (None, 4)                 208       
                                                                 
 dense_11 (Dense)            (None, 1)                 5         
                                                                 
Total params: 3381 (13.21 KB)
Trainable params: 3381 (13.21 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [64]:
with tf.device("/gpu:0"):
    H=model.fit(x_train,y_train,
            validation_data=(x_test, y_test),
            epochs=200,batch_size=32,
            callbacks=callbacks_list,
            verbose=1)

Epoch 1/200


2023-10-12 14:58:39.275865: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:425] Loaded runtime CuDNN library: 8.3.2 but source was compiled with: 8.6.0.  CuDNN library needs to have matching major version and equal or higher minor version. If using a binary install, upgrade your CuDNN library.  If building from sources, make sure the library loaded at runtime is compatible with the version specified during compile configuration.
2023-10-12 14:58:39.276790: W tensorflow/core/framework/op_kernel.cc:1828] OP_REQUIRES failed at cudnn_rnn_ops.cc:1762 : UNKNOWN: Fail to find the dnn implementation.


UnknownError: Graph execution error:

Fail to find the dnn implementation.
	 [[{{node CudnnRNN}}]]
	 [[sequential_11/lstm_29/PartitionedCall]] [Op:__inference_train_function_29808]

## HLS4ML

In [3]:
%%bash
cat /etc/lsb-release

cat: /etc/lsb-release: No such file or directory


CalledProcessError: Command 'b'cat /etc/lsb-release\n'' returned non-zero exit status 1.

In [5]:
import os

os.environ['PATH'] 

'/global/homes/n/nrvora/.local/perlmutter/tensorflow2.12.0/bin:/global/common/software/nersc/pm-2022q4/sw/tensorflow/2.12.0/bin:/global/common/software/nersc/pm-2022q4/sw/evp-patch/bin:/global/homes/n/nrvora/.conda/envs/readout/bin:/global/common/software/nersc/pe/conda/23.9.0/Miniconda3-py311_23.5.2-0/condabin:/global/u1/n/nrvora/.vscode-server/bin/f1b07bd25dfad64b0167beb15359ae573aecd2cc/bin/remote-cli:/opt/nersc/pe/bin:/global/common/software/nersc/bin:/global/common/software/nersc/pm-2021q4/easybuild/software/Nsight-Systems/2022.2.1:/global/common/software/nersc/pm-2021q4/easybuild/software/Nsight-Systems/2022.2.1/bin:/global/common/software/nersc/pm-2021q4/easybuild/software/Nsight-Compute/2022.1.1:/opt/nvidia/hpc_sdk/Linux_x86_64/22.7/cuda/11.7/compute-sanitizer:/opt/nvidia/hpc_sdk/Linux_x86_64/22.7/cuda/11.7/bin:/opt/nvidia/hpc_sdk/Linux_x86_64/22.7/cuda/11.7/libnvvp:/opt/nvidia/hpc_sdk/Linux_x86_64/22.7/profilers/Nsight_Compute:/opt/nvidia/hpc_sdk/Linux_x86_64/22.7/profilers/Ns